In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

In [4]:
import re
import pickle
from pathlib import Path

import gurobipy
from gurobipy import GRB
import dgl
import numpy as np
import torch

from src.problem import get_model, get_soc, load_instance

In [5]:
instances = sorted(list(Path('../data/raw').glob('97_9*.jl')))

i = 1
print(instances[i])
instance = load_instance(instances[i])

J = instance['jobs'][0]
T = instance['tamanho'][0]
uso_p = instance['uso_p']
recurso_p = instance['recurso_p']

jobs = list(range(J))
model = get_model(jobs, instance, coupling=True)

with open('../97_9_opts.pkl', 'rb') as f:
    opts = pickle.load(f)

x_opt = opts[instances[i].name]['sol']
x_opt.shape

../data/raw/97_9_1.jl
Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-10


(1746,)

In [6]:
model_vars = np.core.defchararray.array([v.getAttr(GRB.Attr.VarName) for v in model.getVars()])
model_vars = model_vars[(model_vars.find('x') >= 0) | (model_vars.find('phi') >= 0)]  # drop soc vars
sol = x_opt[model_vars.find('x') == -1]
sol_vars = model_vars[model_vars.find('phi') == -1]

sol_idx = [re.fullmatch(r"x\((\d+),(\d+)\)", s_v).groups() for s_v in sol_vars]
sol_idx = np.array(list(map(lambda jt: list(map(int, jt)), sol_idx)))

In [10]:
def benders_subproblem(instance, solucao, solve=True, verbose=True, get_cut_if_infeasible=True):
    subproblem = gurobipy.Model()
    if verbose:
        subproblem.Params.LogToConsole = 1

    lmbd0 = {}
    lmbd1 = {}
    lmbd2 = {}
    lmbd3 = {}
    lmbd4 = {}
    lmbd5 = {}
    lmbd6 = {}
    lmbd7 = {}

    J = instance['jobs'][0]
    T = instance['tamanho'][0]
    uso_p = instance['uso_p']
    recurso_p = instance['recurso_p']

    soc_inicial = 0.7
    limite_inferior = 0.0
    ef = 0.9
    v_bat = 3.6
    q = 5
    bat_usage = 5

    for t in range(T):
        lmbd1[t] = subproblem.addVar(name="lmbd1(%s)" % t, lb=0, vtype=GRB.CONTINUOUS) # alpha 1 
        lmbd2[t] = subproblem.addVar(name="lmbd2(%s)" % t, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS) # b 2 
        lmbd3[t] = subproblem.addVar(name="lmbd3(%s)" % t, lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS) # b 3
        lmbd4[t] = subproblem.addVar(name="lmbd4(%s)" % t, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS) # soc - i 4
        lmbd5[t] = subproblem.addVar(name="lmbd5(%s)" % t, lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS) # soc - i 5
        lmbd6[t] = subproblem.addVar(name="lmbd6(%s)" % t, lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS) # limite inferior 6
        lmbd7[t] = subproblem.addVar(name="lmbd7(%s)" % t, lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS) # limite inferior 6

    for t in range(T):
        subproblem.addConstr((bat_usage * v_bat)*lmbd1[t] + lmbd7[t] >= 0)
        subproblem.addConstr(lmbd2[t] - lmbd3[t]/v_bat == 0)
        subproblem.addConstr(lmbd3[t] -  (ef / q)*(1/60) * lmbd4[t] == 0)

    for t in range(T-1):
            subproblem.addConstr(lmbd4[t] - lmbd4[t+1] + (-lmbd5[t] + lmbd6[t]) == 0)

    subproblem.addConstr(lmbd4[T-1] + (-lmbd5[T-1] + lmbd6[T-1]) == 0)
    subproblem.setParam('InfUnbdInfo', 1)
    subproblem.Params.LogToConsole = 0
    subproblem.update()

    obj = 0

    for t in range(T):
        lhs = float(sum(uso_p[j] * solucao[j][t] for j in range(J)))
        obj += lmbd1[t] * ((recurso_p[t] + bat_usage * v_bat) - lhs) + lmbd2[t] * (recurso_p[t] - lhs) - 0.0*lmbd5[t] + lmbd6[t] + lmbd7[t]
    obj += lmbd4[0] * 0.7
    subproblem.setObjective(obj, GRB.MINIMIZE)
    subproblem.update()

    if solve:
        subproblem.optimize()

        if get_cut_if_infeasible and subproblem.status == 5:
            teste = gurobipy.Model()
            A = teste.addVars(J,T, vtype=GRB.BINARY, name="A")
            teste.update()
            cut = 0
            for t in range(T):
                lhs = sum(uso_p[j] * A[j,t] for j in range(J))
                cut += (lmbd1[t].getAttr(GRB.Attr.UnbdRay) * ((recurso_p[t] + bat_usage * v_bat) - lhs) + lmbd2[t].getAttr(GRB.Attr.UnbdRay) * (recurso_p[t] - lhs)
                        - lmbd5[t].getAttr(GRB.Attr.UnbdRay)*0.0 + lmbd6[t].getAttr(GRB.Attr.UnbdRay) + lmbd7[t].getAttr(GRB.Attr.UnbdRay) )
            cut += lmbd4[0].getAttr(GRB.Attr.UnbdRay) * 0.7
            cut = str(cut)

            indices = {}
            corte = []
            # Extracting the indices and coefficients using regular expression
            for match in re.finditer(r"([+-]?\d+\.\d+) A\[(\d+),(\d+)\]", cut):
                coefficient = float(match.group(1))
                i = int(match.group(2))
                j = int(match.group(3))
                indices[(i,j)] = float(coefficient)
                #if float(coefficient) != 0:
                #    print(i,j,coefficient)
                #    #print(pato)

            # assuming the cut is of the form w^T x >= b
            cut_w = np.zeros_like(solucao)
            for (i, j), w_ij in indices.items():
                cut_w[i,j] = w_ij
            cut_b = float(cut.split(' ')[0])
            # indices['const'] = cut.split(' ')[0]
            # #corte = [val for val in indices.values()]
            # for j in range(J):
            #     for t in range(T):
            #         corte.append(indices[(j,t)])
            # corte.append(indices['const'])

            return subproblem, (cut_w, cut_b)
        elif subproblem.status != 2:
            print('ERROR: status ', subproblem.status)

    return subproblem

x_cand_opt = np.zeros_like(sol).reshape((J, T))
for sol_jt, (j, t) in zip(sol, sol_idx):
    x_cand_opt[j,t] = sol_jt

benders_subproblem(instance, x_cand_opt).status

Set parameter InfUnbdInfo to value 1


2

In [13]:
x_cand_dummy = np.ones_like(x_cand_opt)

cut_w, cut_b = benders_subproblem(instance, x_cand_dummy)[1]

print((cut_w * x_cand_dummy).sum() + cut_b >= 0)
print((cut_w * x_cand_opt).sum() + cut_b >= 0)

Set parameter InfUnbdInfo to value 1
False
True


Applying Benders by solving the master with Gurobi:

In [26]:
instances = sorted(list(Path('../data/raw').glob('97_9*.jl')))

i = 0
print(instances[i])
instance = load_instance(instances[i])

J = instance['jobs'][0]
T = instance['tamanho'][0]
uso_p = instance['uso_p']
recurso_p = instance['recurso_p']

with open('../97_9_opts.pkl', 'rb') as f:
    opts = pickle.load(f)

x_opt = opts[instances[i].name]['sol']

# solve problem without coupling constraints
model = get_model(jobs, instance, coupling=False, new_ineq=True)
model.setParam('TimeLimit', 960)
model.optimize()
model.status

../data/raw/97_9.jl


2

In [27]:
x_cand = np.zeros((J,T))
for var in model.getVars():
    var_name = var.getAttr(GRB.Attr.VarName)
    try:
        j, t = re.match(r"x\((\d+),(\d+)\)",var_name).groups()
        j = int(j)
        t = int(t)
        x_cand[j,t] = var.X
    except AttributeError:
        continue
cut_w, cut_b = benders_subproblem(instance, x_cand)[1]

print((cut_w * x_cand).sum() + cut_b >= 0)
print((cut_w * x_cand_opt).sum() + cut_b >= 0)

Set parameter InfUnbdInfo to value 1
False
True


In [28]:
model.addConstr(
    sum(cut_w[j,t] * model.getVarByName(f"x({j},{t})")
        for j in range(J) for t in range(T))
    + cut_b >= 0
)
model.update()
# model.setParam('TimeLimit', 960)
model.optimize()
model.status

2

In [29]:
new_x_cand = np.zeros((J,T))
for var in model.getVars():
    var_name = var.getAttr(GRB.Attr.VarName)
    try:
        j, t = re.match(r"x\((\d+),(\d+)\)",var_name).groups()
        j = int(j)
        t = int(t)
        new_x_cand[j,t] = var.X
    except AttributeError:
        continue
benders_subproblem(instance, new_x_cand).status

Set parameter InfUnbdInfo to value 1


2

Benders with feasibility recovery:

In [47]:
def get_feasible(model, incumbent):
    model_ = model.copy()

    expr = 0
    for j in range(J):
        for t in range(T):
            #if model.getVarByName("x(%s,%s,%s)" % (s,j,t)).x > 0.5:
            if incumbent[j][t] > 0.5:
                expr += (1 - model_.getVarByName("x(%s,%s)" % (j,t)))
            else:
                expr += model_.getVarByName("x(%s,%s)" % (j,t))
            #if incumbent_phi[j][t] > 0.5:
            #    expr += (1 - model.getVarByName("phi(%s,%s)" % (j,t)))
            #else:
            #    expr += model.getVarByName("phi[%s,%s,%s]" % (j,t))

    theta = model_.addVar(vtype=GRB.CONTINUOUS, name="theta")
    model_.addConstr(expr <= theta)
    M1 = 1000
    model_.setObjective(M1*theta, GRB.MINIMIZE)
    model_.update()
    model_.optimize()

    if model_.status == 2:
        feas_incumbent = np.zeros_like(incumbent)
        for var in model_.getVars():
            try:
                j, t = re.fullmatch(r"x\((\d+),(\d+)\)", var.getAttr(GRB.Attr.VarName)).groups()
            except AttributeError:
                continue
            j = int(j)
            t = int(t)
            feas_incumbent[j,t] = var.X

    return feas_incumbent

instances = sorted(list(Path('../data/raw').glob('97_9*.jl')))

i = 0
print(instances[i])
instance = load_instance(instances[i])

J = instance['jobs'][0]
T = instance['tamanho'][0]

with open('../97_9_opts.pkl', 'rb') as f:
    opts = pickle.load(f)

x_opt = opts[instances[i].name]['sol']

x_cand_dummy = np.ones_like(x_cand_opt, dtype=int)

# solve problem without coupling constraints
model = get_model(jobs, instance, coupling=False, new_ineq=False)

x_cand_feasible = get_feasible(model, x_cand_dummy)
x_cand_feasible.sum()

../data/raw/97_9.jl


617

In [48]:
cut_w, cut_b = benders_subproblem(instance, x_cand_feasible)[1]

print((cut_w * x_cand_feasible).sum() + cut_b >= 0)
print((cut_w * x_cand_opt).sum() + cut_b >= 0)

Set parameter InfUnbdInfo to value 1
True
True
